<a href="https://colab.research.google.com/github/omid-sakaki-ghazvini/Practices/blob/main/Financial_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Dependencies and Setup

<div style="direction:rtl">
<font color='green' size="5px">
 کتابخانه های مورد نیاز را نصب میکنیم
    </font>
</div>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

<div style="direction:rtl">
<font color='green' size="5px">
 از لینک زیر دیتاست را دانلود کرده و در پوشه هم مسیر همین ژوپیتر نوت بوک قرار دهید
    </font>
</div>

## https://www.kaggle.com/datasets/lorenzozoppelletto/financial-risk-for-loan-approval/data

# 2. Load Data

<div style="direction:rtl">
<font color='green' size="5px">
توسط خط فرمان زیر، دیتا را فراخوانی میکنیم
    </font>
</div>

In [ ]:
df = pd.read_csv('/kaggle/input/financial-risk-for-loan-approval/Loan.csv')
df.head()

In [ ]:
df = df.drop(columns=['ApplicationDate'], axis=1)

In [ ]:
df.info()

In [ ]:
df.describe()

## 3.Data preparation

In [ ]:
le = LabelEncoder()

for column in df.select_dtypes(include=['object']).columns:
    df[column] = le.fit_transform(df[column])

In [ ]:
X = df.drop('LoanApproved', axis=1)
y = df['LoanApproved']

# 4. Split & Scale Data

<div style="direction:rtl">
<font color='green' size="5px">
 نرمالسازی دیتا
    </font>
</div>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

# 5. Build Models

In [ ]:
models = {
    'RF' : RandomForestClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=42)),
    'GradBoost': GradientBoostingClassifier(random_state=42),
    'XGB' : XGBClassifier(random_state=42),
    'LGBM' : LGBMClassifier(random_state=42)

}

results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred, output_dict=True)

    results[name] = {
        'Accuracy': accuracy,
        'Precision': classification_rep['1']['precision'],
        'Recall': classification_rep['1']['recall'],
        'F1': classification_rep['1']['f1-score']
    }

results_df = pd.DataFrame(results).T

# 6. Results

In [ ]:
results_df

In [ ]:
plt.figure(figsize=(10,6))

plt.title("Results")

sns.lineplot(results_df['Accuracy'])

plt.ylabel("Accuracy");